<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Connect/blob/main/Wheel_Strategy_Apple_Options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the required packages
from AlgorithmImports import *

class OptionWheelStrategy(QCAlgorithm):
    # Initialise the class
    def initialize(self):
        self.set_start_date(2022, 1, 1) # Set the start date
        self.set_end_date(2025, 1, 1) # Set the end date
        self.set_cash("USD", 1000000) # Set the strategy cash
        self.set_security_initializer(BrokerageModelSecurityInitializer(self.brokerage_model, FuncSecuritySeeder(self.get_last_known_prices))) # Define our brokerage model
        self._asset = self.add_equity("AAPL", data_normalization_mode=DataNormalizationMode.Raw) # Define the underlying asset which in this case is Apple stock
        self._otm_thresh = 0.1 # Set the out of the money threshold to be 10%

    def on_data(self, data):
        # First check for an existing position in the underlying asset
        if [self._asset.symbol] == [symbol for symbol, holding in self.portfolio.items() if holding.invested]:
            # If so then select an out of the money call and short it
            ticker = self._get_contracts(OptionRight.CALL, self._asset.price * (1+self._otm_thresh))
            self.market_order(ticker, -self._asset.holdings.quantity / 100)

        # Otherwise if the market is open and there is no position in the underlying asset
        elif self.is_market_open(self._asset.symbol) and not self.portfolio.invested:
            # Select an out of the money put and short it
            ticker = self._get_contracts(OptionRight.PUT, self._asset.price * (1-self._otm_thresh))
            self.set_holdings(ticker, -0.2)


    def _get_contracts(self, right, target_price):
        # Returns options contracts based on type, strike price and expiration date
        contract_tickers = self.option_chain_provider.get_option_contract_list(self._asset.symbol, self.time)
        expiry_date = min([x.id.date for x in contract_tickers if x.id.date.date() > self.time.date() + timedelta(60)])
        filtered_tickers = [
            x for x in contract_tickers
            if (x.id.option_right == right and x.id.date == expiry_date and (x.id.strike_price <= target_price if right == OptionRight.PUT else x.id.strike_price >= target_price))
        ]
        ticker = sorted(filtered_tickers, key=lambda x: x.id.strike_price, reverse=right == OptionRight.PUT)[0]
        self.add_option_contract(ticker)
        return ticker

